### Import

In [1]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings(action='ignore')

### Read data

In [2]:
data = pd.read_parquet('../data/DataCleansing.pqt')
data.shape

(929040, 29)

In [3]:
# 2018~2023년 열린 공연 정보를 담은 외부데이터이다.
# https://www.sac.or.kr/site/main/program/schedule

outerPF = pd.DataFrame()
for i in range(2018,2024):
    ease = pd.read_csv(f'../data/Outerdata/예술의전당_공연 및 전시 안내_{i}.csv', encoding='cp949', 
                       usecols = ['장르', '공연/전시명', '기간', '장소'])\
           .query('장소=="콘서트홀"').rename(columns={'공연/전시명':'공연명'})
    # 형식에 어긋난 데이터는 병합하지 않는다.
    ease = ease[ease['기간'].apply(lambda x: len(x)) == 10]
    outerPF = pd.concat([outerPF, ease])

In [4]:
# 취소되지 않은 공연에 대해 분석한다.
outerPF = outerPF[outerPF['공연명'].apply(lambda x: '공연취소' not in x)]

# column을 정리한다.
outerPF['장르'].fillna('클래식', inplace=True)
outerPF['기간'] = pd.to_datetime(outerPF['기간'])
del outerPF['장소']
print('2018~2023년 공연수:', outerPF.shape)

2018~2023년 공연수: (1328, 3)


### Generate performance information

In [5]:
performance = data.drop_duplicates('performance_label').set_index('performance_label').sort_index()\
              [['play_date','play_st_time','genre','pre_open_date','open_date']]
performance

,play_date,play_st_time,genre,pre_open_date,open_date
performance_label,,,,,
0,2022-02-04,2000,교향곡,2022-01-14,2022-01-15
1,2022-03-02,1930,독주,2022-01-09,2022-01-10
2,2019-03-23,2000,교향곡,2018-11-19,2018-11-19
4,2019-07-23,2000,교향곡,2019-06-02,2019-06-02
5,2022-06-29,1930,교향곡,2022-04-30,2022-04-30
...,...,...,...,...,...
688,2021-05-08,1700,교향곡,NaT,2021-01-01
689,2021-09-14,1930,독주,NaT,2021-08-22
690,2021-07-23,1900,성악,2021-06-27,2021-06-28


- play_date, play_st_time으로부터 일자 정보를 생성한다.

In [6]:
performance['play_year'] = performance['play_date'].dt.year
performance['play_month'] = performance['play_date'].dt.month
performance['play_weekday'] = performance['play_date'].dt.weekday
performance['play_time'] = performance['play_st_time']//100 + performance['play_st_time']%100/60

- play_date와 open_date 차이를 구한다.

In [7]:
performance['open_gap'] = (performance['play_date'] - performance['open_date']).dt.days.fillna(-1)

- open_date로부터 일자 정보를 생성한다.

In [8]:
performance['open_year'] = performance['open_date'].dt.year
performance['open_month'] = performance['open_date'].dt.month

- pre_open_date와 open_date 차이를 구해 선예매여부 및 선예매기간을 구한다.

In [9]:
performance['pre_open_gap'] = (performance['open_date'] - performance['pre_open_date']).dt.days.fillna(-1)
performance.drop(['pre_open_date','open_date'], axis=1, inplace=True)
data.drop(['pre_open_date','open_date'], axis=1, inplace=True)

### 공연의 좌석등급별 원가 복원
- 원가(discount_type이 "일반"인 데이터)와 복원한 원가로 공연별 등급제, 등급별 가격을 확인한다.
- 725개 공연 중 628개 공연 정보를 확보했다.

In [10]:
# 천원 단위로 떨어지는 것들을 원가로 생각한다.
ease = data[['performance_label', 'origin_price']]
ease['origin_price_unit'] = ease['origin_price'].apply(lambda x: round(x) if '000.0' in str(x) else np.nan)
performance = performance.merge(ease.dropna(subset=['origin_price_unit'])\
                                .groupby('performance_label')['origin_price_unit'].agg(lambda x: sorted(set(x), reverse=True)).rename('prices').reset_index(),
                                on='performance_label')

# 할인으로 인한 천원 단위 차이를 보완한다.
# 이로써 681개 공연 중 637개 공연의 원가정보를 추가했다.(44개 공연은 좌석별 원가를 알아낼 수 없었다.)
performance['prices'] = performance.prices.apply(lambda x: [x[0]]+[x[i] for i in range(1,len(x)) if x[i-1] - x[i] >= 5000])
performance['n_grade'] = performance['prices'].apply(lambda x: len(x))

In [11]:
# 콘서트홀 규정상 좌석을 단일등급화하거나 2~5등급으로만 쪼갤 수 있다.
# 이로써 가격 정보를 알 수 있는 629개 공연의 좌석별 가격, 등급수, 등급을 알아냈다.
performance = performance.loc[performance.query('n_grade <= 5').index]
performance['grade'] = performance['n_grade'].map({1:['single'],
                                                   2:['R','S'],
                                                   3:['R','S','A'],
                                                   4:['R','S','A','B'],
                                                   5:['R','S','A','B','C']})
performance

,performance_label,play_date,play_st_time,genre,play_year,play_month,play_weekday,play_time,open_gap,open_year,open_month,pre_open_gap,prices,n_grade,grade
0,0,2022-02-04,2000,교향곡,2022,2,4,20.0,20,2022,1,1.0,"[120000.0, 90000.0, 50000.0, 10000.0]",4,"[R, S, A, B]"
1,1,2022-03-02,1930,독주,2022,3,2,19.5,51,2022,1,1.0,"[180000.0, 140000.0, 110000.0, 70000.0]",4,"[R, S, A, B]"
2,2,2019-03-23,2000,교향곡,2019,3,5,20.0,124,2018,11,0.0,"[350000.0, 260000.0, 180000.0, 120000.0, 70000.0]",5,"[R, S, A, B, C]"
3,4,2019-07-23,2000,교향곡,2019,7,1,20.0,51,2019,6,0.0,"[150000.0, 120000.0, 80000.0, 30000.0, 20000.0]",5,"[R, S, A, B, C]"
4,5,2022-06-29,1930,교향곡,2022,6,2,19.5,60,2022,4,0.0,"[80000.0, 60000.0, 20000.0]",3,"[R, S, A]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,688,2021-05-08,1700,교향곡,2021,5,5,17.0,127,2021,1,-1.0,"[80000.0, 60000.0, 30000.0, 10000.0]",4,"[R, S, A, B]"
633,689,2021-09-14,1930,독주,2021,9,1,19.5,23,2021,8,-1.0,"[110000.0, 90000.0, 70000.0]",3,"[R, S, A]"
634,690,2021-07-23,1900,성악,2021,7,4,19.0,25,2021,6,1.0,"[121000.0, 110000.0, 88000.0, 66000.0]",4,"[R, S, A, B]"
635,691,2021-12-12,1700,교향곡,2021,12,6,17.0,36,2021,11,1.0,"[60000.0, 40000.0, 20000.0]",3,"[R, S, A]"


### 공연명 병합
외부데이터를 사용해 공연명을 병합한다.
- 두 개 이상 데이터와 결합된 데이터들 중 장르가 동일한 데이터와 결합된 데이터와 장르가 모두 다른 경우 결측치로 처리한다.

In [12]:
ease = performance.merge(outerPF, left_on='play_date', right_on='기간', how='left')

# 629개 공연 = 154개 공연(두 데이터셋 간 매칭되지 않은 공연) + 442개 공연(정확히 매칭된 공연) 
#             + 19개(2개 데이터와 매칭되어 같은 장르로 구분한 뒤 drop_duplicate한 공연) 
#             + 14개(2개 데이터와 매칭되었으나 같은 장르가 없어 null 처리된 데이터)
mul = ease.performance_label.value_counts()[ease.performance_label.value_counts() >= 2].index
ease_mul = ease.query('performance_label in @mul')

treated = ease_mul.query('genre == 장르 | genre == "클래식"').sort_values(by='공연명').drop_duplicates('performance_label')
performance = pd.concat([ease.query('performance_label not in @mul'), treated,
                         ease_mul.query('performance_label not in @treated.performance_label').drop('공연명', axis=1).drop_duplicates('performance_label')])
performance.drop(['장르','기간'], axis=1, inplace=True)
performance.reset_index(drop=True, inplace=True)
performance

,performance_label,play_date,play_st_time,genre,play_year,play_month,play_weekday,play_time,open_gap,open_year,open_month,pre_open_gap,prices,n_grade,grade,공연명
0,0,2022-02-04,2000,교향곡,2022,2,4,20.0,20,2022,1,1.0,"[120000.0, 90000.0, 50000.0, 10000.0]",4,"[R, S, A, B]",NaN
1,1,2022-03-02,1930,독주,2022,3,2,19.5,51,2022,1,1.0,"[180000.0, 140000.0, 110000.0, 70000.0]",4,"[R, S, A, B]","국립합창단 기획공연 위대한 합창 시리즈Ⅰ- 칼 오르프, 카르미나 부라나"
2,2,2019-03-23,2000,교향곡,2019,3,5,20.0,124,2018,11,0.0,"[350000.0, 260000.0, 180000.0, 120000.0, 70000.0]",5,"[R, S, A, B, C]",오페라 카니발 2019
3,4,2019-07-23,2000,교향곡,2019,7,1,20.0,51,2019,6,0.0,"[150000.0, 120000.0, 80000.0, 30000.0, 20000.0]",5,"[R, S, A, B, C]",제17회 코리아니쉬 플루트 오케스트라 정기연주회
4,5,2022-06-29,1930,교향곡,2022,6,2,19.5,60,2022,4,0.0,"[80000.0, 60000.0, 20000.0]",3,"[R, S, A]",<강남심포니오케스트라 제92회 정기연주회>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,458,2022-11-25,1930,독주,2022,11,4,19.5,96,2022,8,1.0,"[70000.0, 50000.0, 30000.0]",3,"[R, S, A]",NaN
625,540,2021-08-28,1930,성악,2021,8,5,19.5,69,2021,6,1.0,"[150000.0, 120000.0, 100000.0, 70000.0, 50000.0]",5,"[R, S, A, B, C]",NaN
626,634,2022-10-15,1700,독주,2022,10,5,17.0,55,2022,8,1.0,"[110000.0, 90000.0, 70000.0, 50000.0]",4,"[R, S, A, B]",NaN
627,649,2022-08-18,1930,독주,2022,8,3,19.5,60,2022,6,1.0,"[50000.0, 30000.0]",2,"[R, S]",NaN


In [13]:
# 공연명의 특수문자를 제거한다.
performance['공연명전처리'] = performance['공연명'].str.replace(pat=r'[^A-Za-z0-9가-힣\.]',repl=r' ',regex=True)\
                              .str.replace('예술의전당', '').str.replace(r'\s+', ' ').str.strip()
performance

,performance_label,play_date,play_st_time,genre,play_year,play_month,play_weekday,play_time,open_gap,open_year,open_month,pre_open_gap,prices,n_grade,grade,공연명,공연명전처리
0,0,2022-02-04,2000,교향곡,2022,2,4,20.0,20,2022,1,1.0,"[120000.0, 90000.0, 50000.0, 10000.0]",4,"[R, S, A, B]",NaN,NaN
1,1,2022-03-02,1930,독주,2022,3,2,19.5,51,2022,1,1.0,"[180000.0, 140000.0, 110000.0, 70000.0]",4,"[R, S, A, B]","국립합창단 기획공연 위대한 합창 시리즈Ⅰ- 칼 오르프, 카르미나 부라나",국립합창단 기획공연 위대한 합창 시리즈 칼 오르프 카르미나 부라나
2,2,2019-03-23,2000,교향곡,2019,3,5,20.0,124,2018,11,0.0,"[350000.0, 260000.0, 180000.0, 120000.0, 70000.0]",5,"[R, S, A, B, C]",오페라 카니발 2019,오페라 카니발 2019
3,4,2019-07-23,2000,교향곡,2019,7,1,20.0,51,2019,6,0.0,"[150000.0, 120000.0, 80000.0, 30000.0, 20000.0]",5,"[R, S, A, B, C]",제17회 코리아니쉬 플루트 오케스트라 정기연주회,제17회 코리아니쉬 플루트 오케스트라 정기연주회
4,5,2022-06-29,1930,교향곡,2022,6,2,19.5,60,2022,4,0.0,"[80000.0, 60000.0, 20000.0]",3,"[R, S, A]",<강남심포니오케스트라 제92회 정기연주회>,강남심포니오케스트라 제92회 정기연주회
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,458,2022-11-25,1930,독주,2022,11,4,19.5,96,2022,8,1.0,"[70000.0, 50000.0, 30000.0]",3,"[R, S, A]",NaN,NaN
625,540,2021-08-28,1930,성악,2021,8,5,19.5,69,2021,6,1.0,"[150000.0, 120000.0, 100000.0, 70000.0, 50000.0]",5,"[R, S, A, B, C]",NaN,NaN
626,634,2022-10-15,1700,독주,2022,10,5,17.0,55,2022,8,1.0,"[110000.0, 90000.0, 70000.0, 50000.0]",4,"[R, S, A, B]",NaN,NaN
627,649,2022-08-18,1930,독주,2022,8,3,19.5,60,2022,6,1.0,"[50000.0, 30000.0]",2,"[R, S]",NaN,NaN


### 예매데이터 내 원가 복원 및 좌석 정보 추가
- 공연별 정보를 바탕으로 천원 단위로 떨어지지 않은 복원한 가격들을 가까운 등급의 가격으로 대체하고 좌석별 등급을 매긴다.

In [14]:
known_label = performance.loc[performance["prices"].notna()].index
# 초대권을 사용한 데이터까지 포함하면 초대권 사용 데이터에 최소가격이 들어가기에 제외한다.
known = data.loc[data['origin_price'].notna()].query('origin_price != 0 and performance_label in @known_label')

add = []
for _, l, p in known[['performance_label','origin_price']].itertuples():
    diff = list(map(lambda x: abs(x-p), performance.loc[l,'prices']))
    add.append(performance.loc[l, 'prices'][diff.index(min(diff))])
    
known['origin_price'] = pd.Series(add, index=known.index)

In [15]:
add = []
for _, l, p in known[['performance_label','origin_price']].itertuples():
    add.append(performance.loc[l, 'grade'][performance.loc[l, 'prices'].index(p)])
    
known['seat_grade'] = pd.Series(add, index=known.index)
known

,tran_date,tran_time,play_date,play_st_time,price,ticket_cancel,genre,running_time,intermission,free_membership,...,블록,열,좌석번호,seat_label,tran_gap,play_gap,res_time,res_time_rank,res_time_rank_scaled,seat_grade
0,2022-01-14,1512,2022-02-04,2000,10000,1,교향곡,120,15,일반,...,BOX9,None,10,1936,-1,21,720.0,313.0,0.267123,B
1,2020-01-16,38,2020-02-11,1930,30000,0,클래식,120,15,일반,...,BOX9,None,10,1936,24,26,2299080.0,836.0,0.397052,B
4,2023-04-29,1322,2023-05-23,1930,24000,0,클래식,150,15,싹틔우미,...,BOX9,None,10,1936,62,24,5437320.0,1678.0,0.728814,A
5,2019-08-24,959,2019-08-28,2000,22000,0,성악,120,20,일반,...,BOX9,None,10,1936,40,4,3092220.0,2048.0,0.986031,R
6,2022-06-24,1406,2022-08-30,1930,18000,1,클래식,120,15,노블,...,BOX9,None,10,1936,-1,67,300.0,64.0,0.031850,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929034,2019-01-07,1256,2019-01-11,2000,35000,0,클래식,105,15,None,...,A,8,13,1363,28,4,2455020.0,1170.0,0.990678,A
929035,2019-04-18,1028,2019-07-02,2000,75000,1,클래식,120,20,일반,...,A,8,12,1362,109,75,9505680.0,1746.0,0.731043,R
929036,2019-02-23,1433,2019-03-01,2000,5000,0,교향곡,120,15,일반,...,A,8,12,1362,41,6,3645120.0,818.0,0.869149,single
929038,2019-02-16,1055,2019-02-28,2000,15000,0,교향곡,120,15,None,...,A,8,12,1362,76,12,6555240.0,737.0,0.819599,B


In [16]:
# 알려진 공연의 초대권 사용 데이터를 병합한다.
known = pd.concat([known, data.query('origin_price == 0 and performance_label in @known_label')])

## Concat data
공연별 정보, 좌석별 정보를 추가한다. 이후 300개 이상의 예매/취소 건수가 있는 공연만 분석 대상으로 한다.

In [17]:
# 좌석별 정보 병합
data = pd.concat([data.drop('origin_price', axis=1), known[['origin_price','seat_grade']]], axis=1)
data

,tran_date,tran_time,play_date,play_st_time,price,ticket_cancel,genre,running_time,intermission,free_membership,...,열,좌석번호,seat_label,tran_gap,play_gap,res_time,res_time_rank,res_time_rank_scaled,origin_price,seat_grade
0,2022-01-14,1512,2022-02-04,2000,10000,1,교향곡,120,15,일반,...,None,10,1936,-1,21,720.0,313.0,0.267123,10000.0,B
1,2020-01-16,38,2020-02-11,1930,30000,0,클래식,120,15,일반,...,None,10,1936,24,26,2299080.0,836.0,0.397052,30000.0,B
2,2019-09-09,1253,2019-10-15,2000,0,0,클래식,100,15,None,...,None,10,1936,14,36,769320.0,751.0,0.345781,0.0,NaN
3,2019-03-08,1447,2019-03-22,2000,0,0,교향곡,90,20,None,...,None,10,1936,5,14,435480.0,1705.0,0.686820,0.0,NaN
4,2023-04-29,1322,2023-05-23,1930,24000,0,클래식,150,15,싹틔우미,...,None,10,1936,62,24,5437320.0,1678.0,0.728814,30000.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929035,2019-04-18,1028,2019-07-02,2000,75000,1,클래식,120,20,일반,...,8,12,1362,109,75,9505680.0,1746.0,0.731043,140000.0,R
929036,2019-02-23,1433,2019-03-01,2000,5000,0,교향곡,120,15,일반,...,8,12,1362,41,6,3645120.0,818.0,0.869149,10000.0,single
929037,2019-03-26,1650,2019-03-30,1700,15000,0,성악,120,20,None,...,8,12,1362,59,4,5107800.0,1334.0,1.000000,NaN,NaN
929038,2019-02-16,1055,2019-02-28,2000,15000,0,교향곡,120,15,None,...,8,12,1362,76,12,6555240.0,737.0,0.819599,30000.0,B


In [18]:
up300 = data.performance_label.value_counts().loc[data.performance_label.value_counts() >= 300].index
data = data.query('performance_label in @up300')
print(f'최종 데이터 크기: {data.shape}')

최종 데이터 크기: (924321, 28)


In [19]:
performance = performance.reset_index().query('performance_label in @up300').set_index('performance_label').sort_index()
print(f'최종 공연수: {performance.shape[0]}')

최종 공연수: 603


## Save data

In [20]:
# 분석에 불필요한 변수 제거 후 저장한다.
data.drop(['play_date','play_st_time'], axis=1, inplace=True)
data.to_parquet('../data/DataCleansing.pqt')

In [21]:
# 분석에 불필요한 변수 제거 후 저장한다.
performance.drop(['play_date','play_st_time'], axis=1, inplace=True)
performance.to_parquet('../data/performance_data.pqt')